<a href="https://colab.research.google.com/github/hrushikeshrv/instagram-data-analysis/blob/master/Instagram_Data_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Hi! Ignore the cell below

In [ ]:
#Imports
import pandas as pd
import re
import string
import matplotlib.pyplot as plt
import seaborn as sns

#Import helper functions
from instagram_utils import find_person, print_conversation, cleaned, frequency_analysis, frequency_plot
#Get the data into a DataFrame
dms = pd.read_json('messages.json')

### Enter your instagram username

In [ ]:
my_username = ''

### Run this cell to see all the usernames you have a chat history with

In [ ]:
all_unames = []
for i in dms['participants']:
    i.pop(i.index(my_username))
    for j in i:
        if j not in all_unames:
            all_unames.append(j)

for x in all_unames:
    print(x)

### Enter the instagram username of the person you want to analyze your history with
(Between the quotes)

In [ ]:
person = ''

## Run this cell to print the entire conversation with the person you chose
### This can be long

In [ ]:
posts_sent, posts_recieved, photos_sent, photos_recieved = print_conversation(person, my_username, dms, True)

## Run this cell to plot a graph of the number of texts sent and recieved vs time

In [ ]:
frequency_plot(person, dms, 3)
frequency_plot(person, dms, 6)

## Run this cell to print a list of the top 20 most common words in your chat with this person

- Replace the 'top' with 'search' and you will be able to search for a particular word and see how many times it was used
- Change the 20 to change the number of top words that are printed

In [ ]:
frequency_analysis(person, dms, 20, 'top')

## NOTE - The below functions are not 100% accurate yet.
#### Run this cell to see all the posts you sent to the person you have selected

In [ ]:
for i in posts_sent:
    print(i)

#### Run this cell to see all the posts you have recieved from the person you have selected

In [ ]:
for i in posts_recieved:
    print(i)

#### Run this cell to see all the non disappearing photos you have sent to the person you have selected

In [ ]:
for i in photos_sent:
    print(i)

#### Run this cell to see all the non disappearing photos you have recieved from the person you have selected

In [ ]:
for i in photos_recieved:
    print(i)

# To look for another person, change the username in the 4th cell and run the cells below it again